In [9]:
import os
import xlwings
import numpy as np
import pandas as pd 

In [10]:
# 常量设置
RATIO_USUAL_SCORE = 0.2  # 平时成绩比例
RATIO_EXPERIMENT = 0.2   # 实验成绩比例
RATIO_FINAL_SCORE = 0.6  # 期末成绩比例
RATIO_SCORE_CLASS= RATIO_FINAL_SCORE

In [11]:
Qimo_table = pd.read_excel('2022春_B机制191-192流体力学基础与液压气动技术-V2-20221108.xlsx', 
                                   index_col=None, header=None, sheet_name='期末考试')

# 获得表格分数数据
def Get_Score_Table_info(Score_Table_Class):
      # 分数表格-类别 
    # print(Score_Table_Class)  

    Course_target_class = np.array(Score_Table_Class.iloc[5,4:-1])  # 课程目标类别
    # print(course_target_class) 
    Student_score = np.array(Score_Table_Class.iloc[8:,4:-1]) # 学生分数（各题）
    # Student_score[:,0]
    Student_info = np.array(Score_Table_Class.iloc[8:,2:4])       # 学生信息 

    Score_Table_Head = Score_Table_Class.iloc[:8,:] # 表头数据

    # 课程目标 类别 
    loc_target_Numall=np.array([])
    loc_target_one = np.argwhere(Course_target_class==1).reshape((-1)) # 满足条件的索引值 
    loc_target_two = np.argwhere(Course_target_class==2).reshape((-1)) # 满足条件的索引值 
    loc_target_three = np.argwhere(Course_target_class==3).reshape((-1)) # 满足条件的索引值 
    # print(loc_target_one,loc_target_two,loc_target_three)
    loc_target_Numall=[loc_target_one, loc_target_two, loc_target_three] 
    print('loc_target_Numall', loc_target_Numall) 

    # 每题满分 
    Fullscore_each_examination_question = np.array((Score_Table_Class.iloc[6,4:-1])).reshape((-1)) 
    # print(Fullscore_each_examination_question)
    return Student_score, Fullscore_each_examination_question, loc_target_Numall, Student_info, Course_target_class, Score_Table_Head


In [12]:
# 达成度目标=NUM 计算
### 数据索引
# 学生分数
def Achievement_Degree_Calcultion_function(Student_score, Fullscore_each_examination_question, 
                                           loc_target_Num, RATIO_SCORE_CLASS):  
    scorelist_student_target_Num=[] 

    for i in range(len(loc_target_Num)): 
        scorelist_student_target_Num.append( Student_score[:,loc_target_Num[i]] ) 
    scorearr_target_Num = np.array(scorelist_student_target_Num).T 
    # print('学生分数', scorearr_target_Num) 

    # 每题题目分数 
    FullscoreEEQ_list_target_Num=[] 
    for i in range(len(loc_target_Num)):   
        FullscoreEEQ_list_target_Num.append(Fullscore_each_examination_question[loc_target_Num[i]] )  

    # print('每题题目分数', FullscoreEEQ_list_target_Num)

    ### 数据计算
    # 实际满分(百分制) - 比例满分 - 学生得分 - 得分比例 - 目标1得分
    RealFullscore_target_Num_Total = sum(FullscoreEEQ_list_target_Num) / np.sum(Fullscore_each_examination_question)  *100  # 实际满分(百分制)
    print('实际满分(百分制)', RealFullscore_target_Num_Total)
    # 比例满分 (乘以比例后的分数) 
    RatioFullscore_target_Num = RealFullscore_target_Num_Total * RATIO_SCORE_CLASS 
    # 学生得分
    Score_student_target_Num_Total = np.sum(scorearr_target_Num, axis=1) 
    # 得分比例
    Ratio_score_target_Num = np.around((Score_student_target_Num_Total/RealFullscore_target_Num_Total).astype(float) ,decimals=4)
    # print( Ratio_score_target_Num ) 
    # 目标1得分 = 比例满分 * 得分比例 
    Score_target_Num =np.around( ( RatioFullscore_target_Num * Ratio_score_target_Num).astype(float) ,decimals=4)
    # print(Score_target_Num)
    
    return np.c_[Score_student_target_Num_Total, Ratio_score_target_Num, Score_target_Num]
    

In [13]:
# 获得新的表头
def Score_Table_Head_Deal(Score_Table_Head):
    w_table = Score_Table_Head.shape[1]
    Table_Head = Score_Table_Head
    # print(w_table)
    new_col = Table_Head.columns.to_list() + list(range(w_table,w_table+3*3+2))
    Table_Head = Table_Head.reindex(columns=new_col)
    namecol_add = ['1', '1°','1得分','2', '2°','2得分','3', '3°','3得分', '计算成绩', '实际成绩']

    Table_Head.iloc[1,w_table:] = namecol_add
    Table_Head.drop(columns=[0,1],axis=1,inplace=True)
    Table_Head.columns = list(range(0,Table_Head.shape[1]))
    return Table_Head

In [119]:
# 整体平均分 + 各项学生得分占比
def AverageScore_ScoreRatio(DataScore_Cal_CourseClass_Num, Fullscore_each_examination_question):
    Score_Average =  np.around(np.mean(DataScore_Cal_CourseClass_Num[:,2:],axis = 0).astype(float),decimals=2) 
    # print(Score_Average) 
    FS_EEQ_T = np.r_[Fullscore_each_examination_question, np.sum(Fullscore_each_examination_question)]
    print(FS_EEQ_T.shape) 
    W_scoreratio = np.around( (Score_Average[:-11]/FS_EEQ_T ).astype(float) , decimals=2 )
    print(W_scoreratio) 
    Score_Average_str = np.r_[np.nan, np.array('平均分'), Score_Average ] 
    W_scoreratio_str  = np.r_[np.nan, np.array('得分率'), W_scoreratio,  np.full((11),np.nan) ] 
    Score_Average_str = Score_Average_str.reshape(-1, len(Score_Average_str))
    W_scoreratio_str  = W_scoreratio_str.reshape(-1, len(W_scoreratio_str))
    return Score_Average_str, W_scoreratio_str

In [121]:
# 自动载入  '成绩表' 


# 主函数 
# 获得表格分数数据 
Student_score, Fullscore_each_examination_question, loc_target_Numall, Student_info, Course_target_class, Score_Table_Head \
           = Get_Score_Table_info(Score_Table_Class = Qimo_table) 

# 计算达成度数据
for TargetNum in [1,2,3]: 
    res = Achievement_Degree_Calcultion_function(Student_score, Fullscore_each_examination_question, 
                                            loc_target_Numall[TargetNum-1], RATIO_SCORE_CLASS) 
    if TargetNum==1:
        Calresult_target_Numall = np.c_[res]
    elif TargetNum>1:
        Calresult_target_Numall = np.c_[Calresult_target_Numall, res]                                        
# print(Calresult_target_Numall) 

# 计算总分 + 实际总分
ScoreTotal_Cal_CourseClass_Num = np.around( np.sum(Calresult_target_Numall[:,2::3] , axis=1).astype(float), decimals=2) 
# print(ScoreTotal_Cal_CourseClass_Num )    
ScoreTotal_Real_CourseClass_Num = np.around( (np.sum(Student_score, axis=1) * RATIO_SCORE_CLASS).astype(float), decimals=2) 
# print(ScoreTotal_Real_CourseClass_Num)  

# 拼接所有数据-1
DataScore_Cal_CourseClass_Num = np.c_[Student_info, Student_score, np.sum(Student_score, axis=1) ,np.around(Calresult_target_Numall.astype(float), decimals=2), \
                                 ScoreTotal_Cal_CourseClass_Num, ScoreTotal_Real_CourseClass_Num]

Score_Average_str, W_scoreratio_str = AverageScore_ScoreRatio(DataScore_Cal_CourseClass_Num, Fullscore_each_examination_question)

# 拼接所有数据-2
DataScore_Cal_all_CourseClass_Num = np.r_[DataScore_Cal_CourseClass_Num, Score_Average_str, W_scoreratio_str]
print(DataScore_Cal_all_CourseClass_Num)

# 最终结果表格写入excel
# 数据表头 
Final_Table_Head = Score_Table_Head_Deal(Score_Table_Head=Score_Table_Head)
# 拼接表头与数据
DataScore_Cal_df=pd.DataFrame(DataScore_Cal_all_CourseClass_Num)
Result_table_Final = pd.concat([Final_Table_Head,DataScore_Cal_df])
with pd.ExcelWriter('课程达成度评价结果.xlsx', mode='w') as writer:
    Result_table_Final.to_excel(writer, sheet_name='期末考试1',header=False,index=False,startrow=0, startcol=0)


loc_target_Numall [array([ 0,  1,  2,  3, 10], dtype=int64), array([ 4,  5, 11], dtype=int64), array([ 6,  7,  8,  9, 12], dtype=int64)]
实际满分(百分制) 34.0
实际满分(百分制) 22.0
实际满分(百分制) 44.0
(14,)
[0.98 1.   1.   0.99 1.   0.98 1.   1.   1.   1.   0.96 0.83 0.75 0.92]
[['1710101202' '庞德放' 6 ... 23.4 57.0 57.0]
 ['190701011002' '赵文弢' 6 ... 22.2 55.8 55.8]
 ['190701011003' '潘淼' 6 ... 22.2 54.0 54.0]
 ...
 ['190701011149' '罗治华' 6 ... 22.8 56.4 56.4]
 [nan '平均分' 5.86 ... 23.36 55.48 55.48]
 [nan '得分率' 0.98 ... nan nan nan]]
